In [1]:
import pandas as pd
import numpy as np
import os
import string
from fuzzywuzzy import fuzz, process
import sklearn
import matplotlib.pyplot as plt

from spellchecker import SpellChecker

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

/Users/kchua/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Cleaning Data

In [2]:
central_data = pd.read_csv('/Users/kchua/Documents/Data_Stories/2023-10-tx-produced-water-spills/data/central_cleaned.csv')
central_data.head()

,dist,district_edit,date_called_in,date_called_in_edit,date_of_spill,date_of_spill_edit,spill_number,rrc_job_number,operator_rp,operator_edit,...,cleanup_oversight_austin,status,comments,compliance_date,file,sheet,cleanup_criteria,cleanup_oversight,rrc_job_number_2,my_of_spill
0,04,4,2013-01-13,2013-01-13,2013-01-13,2013-01-13,04-1238,2013-381,Headington Oil Company,HEADINGTON OIL,...,NaN,assessment phase,Leak in gathering line DS,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01
1,06,6,2013-01-06,2013-01-06,2013-01-05,2013-01-05,1291,2013-06-195,"Quantum Resources Management, LLC",QUANTUM RESOURCES,...,41284,JH,NaN,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01
2,7C,7C,2013-01-22,2013-01-22,2013-01-22,2013-01-22,1829,324,EOG Resources,EOG Resources,...,No,assessment phase,Dump valve misfired,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01
3,7C,7C,2013-01-14,2013-01-14,2013-01-13,2013-01-13,1830,191,Pioneer Natural Res. USA Inc.,Pioneer Natural Resources (acq. by ExxonMobil ...,...,No,cleanup phase,"water leg plugged up water went in oil tank, r...",NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01
4,7C,7C,2013-01-14,2013-01-14,2013-01-05,2013-01-05,1832,192,"Molopo Energy Texas, LLC","MOLOPO ENERGY TEXAS, LLC",...,No,cleanup phase,heater treater was not started ran prod tank over,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01


In [3]:
data = central_data.copy(deep=True)
print(data.shape)
data.head()

(10326, 56)


,dist,district_edit,date_called_in,date_called_in_edit,date_of_spill,date_of_spill_edit,spill_number,rrc_job_number,operator_rp,operator_edit,...,cleanup_oversight_austin,status,comments,compliance_date,file,sheet,cleanup_criteria,cleanup_oversight,rrc_job_number_2,my_of_spill
0,04,4,2013-01-13,2013-01-13,2013-01-13,2013-01-13,04-1238,2013-381,Headington Oil Company,HEADINGTON OIL,...,NaN,assessment phase,Leak in gathering line DS,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01
1,06,6,2013-01-06,2013-01-06,2013-01-05,2013-01-05,1291,2013-06-195,"Quantum Resources Management, LLC",QUANTUM RESOURCES,...,41284,JH,NaN,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01
2,7C,7C,2013-01-22,2013-01-22,2013-01-22,2013-01-22,1829,324,EOG Resources,EOG Resources,...,No,assessment phase,Dump valve misfired,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01
3,7C,7C,2013-01-14,2013-01-14,2013-01-13,2013-01-13,1830,191,Pioneer Natural Res. USA Inc.,Pioneer Natural Resources (acq. by ExxonMobil ...,...,No,cleanup phase,"water leg plugged up water went in oil tank, r...",NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01
4,7C,7C,2013-01-14,2013-01-14,2013-01-05,2013-01-05,1832,192,"Molopo Energy Texas, LLC","MOLOPO ENERGY TEXAS, LLC",...,No,cleanup phase,heater treater was not started ran prod tank over,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01


## Functions

In [4]:
# Cleaning and fuzzy matching the source values

# Remove punctuation and whitespace
def remove_punctuation(text):
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

def remove_whitespace(text):
    text = ' '.join(text.split())
    return text

# Spell checking
spell = SpellChecker()

def correct_spelling(x, speller=spell):
    words = spell.split_words(x)
    word_list = [spell.correction(word) if not word.isupper() else word for word in words]
    return " ".join(word_list)

# Lowercasing
def impute_nulls(col): 
    clean_col = col.fillna('null')
    clean_col = clean_col.replace('', 'null')
    clean_col = clean_col.apply(lambda x: x.strip())
    clean_col = clean_col.apply(remove_whitespace)
    return clean_col

# Functions for typo correction
def correct_typos(x, typo_dict): 
    if x in typo_dict: 
        return typo_dict[x]
    else: 
        return x

In [5]:
correct_spelling('SWD')

'SWD'

In [6]:
# Functions for typo exploration

def find_best_match(value, choices, threshold=80):
    match, score = process.extractOne(value, choices)
    return match if score >= threshold else value

def plot_kcluster_elbow(word_list):
    # Create the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(word_list)

    # Experiment with a range of clusters
    max_clusters = 10
    distortions = []

    for i in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=i, random_state=42)
        kmeans.fit(X)
        distortions.append(kmeans.inertia_)

    # Plot the Elbow curve
    plt.plot(range(1, max_clusters + 1), distortions, marker='o')
    plt.title('Elbow Method for Optimal k')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Distortion')
    plt.show()

def show_kclusters(word_list, optimal_clusters):
    # Create the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(word_list)

    kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
    kmeans.fit(X)

    word_list_clusters = list(zip(word_list, kmeans.labels_))

    for cluster_id in range(optimal_clusters):
        cluster_words = [word for word, label in word_list_clusters if label == cluster_id]
        print(f"Cluster {cluster_id + 1}: {cluster_words}")
        print()
        print()

## Column Record

In [7]:
# Reason-related columns
reason_cols = [
'type_operation',
'source', 
'probable_cause_edit',
'comments',
]

# RRC columns
rrc_cols = [
'rrc_job_number', 
'rrc_id_number',
'rrc_job_number_2'
]

# Location-related columns
loc_cols = [
'district_edit', 
'operator_edit',
    
'lease_facility_name',
'county_edit', 
'affected_area', 
'spill_on_water_edit'
]

# Whistleblower/reporting/regulations-specific columns 
rep_cols = [
'date_called_in_edit',
'date_of_spill_edit',
'spill_number', 
'rrc_job_number',
'basis', 
'affected_area', 
'swr_20', 
'swr_98exempt',
'cleanup_criteria', 
'cleanup_criteria_swr_91', 
'cleanup_criteria_7_00_doc', 
'cleanup_criteria_case_specific', 
'form_h_8rqrd_edit',
'date_h_8rcvd', 
'cleanup_oversight', 
'cleanup_oversight_district', 
'cleanup_oversight_austin', 
'status', 
'compliance_date'
]

## Cleaning reason_cols - 2/4

In [8]:
# EDA of the nullage
complete_log = [1, 1, 1, 1]
for col, complete in zip(reason_cols, complete_log): 
    print(col)
    print('sum of nulls:', data[col].isnull().sum())
    print('coverage:', ((len(data)-data[col].isnull().sum())/ len(data)) * 100)
    print('unique values:', data[col].nunique())
    if complete == 1: 
        print('!!! COMPLETE !!!')
    print()

type_operation
sum of nulls: 49
coverage: 99.52546968816578
unique values: 74
!!! COMPLETE !!!

source
sum of nulls: 59
coverage: 99.42862676738329
unique values: 255
!!! COMPLETE !!!

probable_cause_edit
sum of nulls: 108
coverage: 98.9540964555491
unique values: 249
!!! COMPLETE !!!

comments
sum of nulls: 1023
coverage: 90.09296920395118
unique values: 6875
!!! COMPLETE !!!



In [9]:
# DONE
col_name = 'type_operation'

# Manual values for type_operation
typo_corrections = {
    'inj/disposal': 'injection/disposal'
}

target_labels = [
    'transportation', 
    'injection/disposal', 
    'disposal',
    'injection',
    'transportation/disposal',
    'production',
    'gathering',
    'completion', 
    # plant for plant, gas plant ? 
    # tank for tank farm, tank battery ?
    # salt water station, brine facility ?
]

col_threshold = 75

# Cleaning process
new_col_name = col_name + '_clean'

data[new_col_name] = data[col_name].astype(str)
data[new_col_name] = impute_nulls(data[new_col_name]).apply(lambda x: x.lower())
data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))

# Check discrepancies
discrepancies = data[[col_name, new_col_name]]
discrepancies = discrepancies[discrepancies[col_name] != discrepancies[new_col_name]]
discrepancies.drop_duplicates()

,type_operation,type_operation_clean
120,NaN,nan
128,Production,production
129,Other,other
339,SWD,swd
609,Completion,completion
620,Disposal,disposal
864,Pipeline,pipeline
1084,Drilling,drilling
1182,Plant,plant
1578,DISPOSAL,disposal


In [10]:
data['type_operation_clean'].value_counts()

type_operation_clean
production                       7333
injection/disposal               2122
commercial disposal               205
transportation                    196
other                             106
drilling                           79
nan                                49
injection                          39
processing                         35
tank farm                          35
transport                          35
disposal                           18
pipeline                           10
gathering                           9
swd                                 5
processing facility                 3
fracking                            3
completions                         2
injection/ disposal                 2
tank battery                        2
plant                               2
commingle                           2
gas plant                           2
gathering line                      2
storage                             2
saltwater station            

In [11]:
# Column source - DONE
col_name = 'source'

typo_corrections = {
    'high prressure separator': 'high pressure separator',
    'hp separator': 'high pressure separator',
    '?': 'unknown',
    'pipline leak': 'pipeline leak', 
    'pl riser': 'pipeline riser', # a likely guess
    'pileline': 'pipeline'
}

target_labels = [
    'tank battery',
    'vacuum truck', 
    'flowline',
    'gathering line',
    'tank',
    'truck',
]

col_threshold = 90
new_col_name = col_name + '_clean'

# Cleaning process
data[new_col_name] = data[col_name].astype(str)
data[new_col_name] = impute_nulls(data[new_col_name]).apply(lambda x: x.lower())
data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))

# Check discrepancies
discrepancies = data[[col_name, new_col_name]]
discrepancies = discrepancies[discrepancies[col_name] != discrepancies[new_col_name]]
discrepancies.drop_duplicates()

,source,source_clean
72,NaN,nan
128,Frac Tanks,frac tanks
129,Tanker Truck,tanker truck
130,TANKS,tanks
203,Tank,tank
...,...,...
10220,HAMMER UNION,hammer union
10222,DUMP ON SEPARATOR,dump on separator
10228,Total Fluid Line,total fluid line
10231,Vehicle Collision,vehicle collision


In [12]:
data['source_clean'].value_counts().head(15)

source_clean
tank battery      4721
flowline          2595
well              1092
other              398
pipeline           286
pump               271
gathering line     269
tank                65
pit                 59
heater treater      59
nan                 59
vehicle             41
separator           27
unknown             26
plant               21
Name: count, dtype: int64

In [13]:
# Column probable_cause_edit
col_name = 'probable_cause_edit'

typo_corrections = {}

new_col_name = col_name + '_clean'

# Cleaning process
data[new_col_name] = data[col_name].astype(str)
data[new_col_name] = impute_nulls(data[new_col_name]).apply(lambda x: x.lower())
data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))

# Check discrepancies
discrepancies = data[[col_name, new_col_name]]
discrepancies = discrepancies[discrepancies[col_name] != discrepancies[new_col_name]]
discrepancies.drop_duplicates()

,probable_cause_edit,probable_cause_edit_clean
0,CORROSION,corrosion
2,MECHANICAL FAILURE,mechanical failure
11,HUMAN ERROR,human error
21,WEATHER,weather
128,Leaking tank,leaking tank
...,...,...
10205,FRAC INTERFERENCE,frac interference
10213,POSSIBLY FRAC'D INTO,possibly frac'd into
10215,FLOWLINE,flowline
10216,FRACKING (NEARBY),fracking (nearby)


In [14]:
# Exploration for typos
#unique_vals = data[new_col_name].unique()
#plot_kcluster_elbow(unique_vals)
#show_kclusters(unique_vals, 10)

In [15]:
# Column comments
col_name = 'comments'

typo_corrections = {
}

new_col_name = col_name + '_clean'

# Cleaning process
data[new_col_name] = data[col_name].astype(str)
data[new_col_name] = impute_nulls(data[new_col_name]).apply(lambda x: x.lower())
data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))

# Check discrepancies
discrepancies = data[[col_name, new_col_name]]
discrepancies = discrepancies[discrepancies[col_name] != discrepancies[new_col_name]]
discrepancies.drop_duplicates()

,comments,comments_clean
0,Leak in gathering line DS,leak in gathering line ds
1,NaN,nan
2,Dump valve misfired,dump valve misfired
6,Spill inside metal containment area at tank ba...,spill inside metal containment area at tank ba...
8,Flowline ruptured due to corrosion,flowline ruptured due to corrosion
...,...,...
10314,Broken flowline,broken flowline
10320,Injection line failed causing spill,injection line failed causing spill
10322,Malfunction at tank caused spill,malfunction at tank caused spill
10324,Spill into North Fork Little Wichita River. Op...,spill into north fork little wichita river. op...


In [16]:
# Exploration for typos
#unique_vals = data[new_col_name].unique()
#plot_kcluster_elbow(unique_vals)
#show_kclusters(unique_vals, 10)

## Clean loc_cols - 4/6

In [17]:
# EDA of the nullage
complete_log = [1, 0, 0, 1, 1, 1]
for col, complete in zip(loc_cols, complete_log): 
    print(col)
    print('sum of nulls:', data[col].isnull().sum())
    print('coverage:', ((len(data)-data[col].isnull().sum())/ len(data)) * 100)
    print('unique values:', data[col].nunique())
    if complete == 1: 
        print('!!! COMPLETE !!!')
    print()

district_edit
sum of nulls: 0
coverage: 100.0
unique values: 13
!!! COMPLETE !!!

operator_edit
sum of nulls: 1
coverage: 99.99031570792175
unique values: 1160

lease_facility_name
sum of nulls: 72
coverage: 99.30273097036607
unique values: 6651

county_edit
sum of nulls: 0
coverage: 100.0
unique values: 189
!!! COMPLETE !!!

affected_area
sum of nulls: 83
coverage: 99.19620375750533
unique values: 50
!!! COMPLETE !!!

spill_on_water_edit
sum of nulls: 765
coverage: 92.59151656013945
unique values: 12
!!! COMPLETE !!!



In [18]:
# Column county_edit - DONE
col_name = 'county_edit'

typo_corrections = {
    'chldress': 'childress',
    'ecor': 'ector'
}

# Cleaning process
new_col_name = col_name + 'clean'
data[new_col_name] = impute_nulls(data[col_name]).apply(lambda x: x.lower())
data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))
data[new_col_name].value_counts()

county_editclean
midland      658
reeves       603
martin       586
andrews      529
ector        509
            ... 
mason          1
coleman        1
gillespie      1
county         1
mcculloch      1
Name: count, Length: 187, dtype: int64

In [19]:
# DONE - new boolean columns
col_name = 'affected_area'

aa_categories = {
    # Are these RRC categories? Are these internal to operators?
    '1': '1= on land, non-sensitive area;',
    '2': '2= on land, sensitive area;', 
    '3': '3= dry waterway;',
    '4': '4= wet/flowing waterway;',
    '5': '5',
    '6': '6',
    '7': '7'
}

typo_corrections = {
    # Adding number labels, if possible
    '?': 'unknown',
    'land--non-sensitive area': '1 land--non-sensitive area',
    'land--sensitive area': '2 land--sensitive area',
    'wet/flowing waterway': '4 wet/flowing waterway', 
    'dry waterway': '3 dry waterway',
    'land --sensitive area': '2 land --sensitive area'
}

def create_boolean_columns(df, column_name, category_dict):
    # Iterate over the categories in the dictionary
    for key, value in category_dict.items():
        # Create a new column with the category name and initialize with False
        df[f'affected_area_{key}_clean'] = 0
        # Update the column based on the presence of the category in the specified column
        df.loc[df[column_name].str.contains(key), f'affected_area_{key}_clean'] = 1

In [20]:
# Manual checking process
data['affected_area'].value_counts()

affected_area
1                                 6002
Land--Non-sensitive Area          2208
1= on land, non-sensitive area     559
VERBAL                             365
2                                  297
Verbal                             249
4                                  148
Land--Sensitive Area                87
3                                   67
EMAIL                               52
Wet/Flowing Waterway                42
2=on land, sensitive area.          32
Dry Waterway                        18
Email                               16
2, 4                                12
2,4                                 12
H-8                                 10
verbal                               7
5                                    7
Private Pond                         7
1, 4                                 4
3=dry  waterway                      4
1, 2                                 3
1, 3                                 3
4=Wet/flowing waterway               3
1,4        

In [21]:
# Check for string values that contain numbers
vals_with_numbers = data[data['affected_area'].str.contains('\d', na=False)]['affected_area'].unique()
print(vals_with_numbers)

['1' '1= on land, non-sensitive area' '3' '2' '4'
 '2=on land, sensitive area.' '5' '7' '3, 2' 'H-8' '3=dry  waterway'
 '4=Wet/flowing waterway' '1 & 2' '1&4' '1, 3' '2,4' '2, 3' '6' '1,3'
 '1, 4' '1,4' '2, 4' '2, 3,4' '4 (private pond-contained)'
 '1 (contianed in concrete TB)' '1, 2, 3' '1, 3, 4' '1, 2' '1, 2, 4']


In [23]:
# Cleaning process
new_col_name = col_name + '_clean'
data[new_col_name] = impute_nulls(data[col_name]).apply(lambda x: x.lower())

data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))

create_boolean_columns(data, new_col_name, aa_categories)
new_aa_columns = [i for i in data.columns.tolist() if 'affected_area' in i]

In [26]:
data[new_aa_columns].sample(15)

,affected_area,affected_area_clean,affected_area_1_clean,affected_area_2_clean,affected_area_3_clean,affected_area_4_clean,affected_area_5_clean,affected_area_6_clean,affected_area_7_clean
5039,1,1,1,0,0,0,0,0,0
4918,1,1,1,0,0,0,0,0,0
631,1,1,1,0,0,0,0,0,0
8041,Land--Non-sensitive Area,1 land--non-sensitive area,1,0,0,0,0,0,0
4003,1,1,1,0,0,0,0,0,0
2421,1,1,1,0,0,0,0,0,0
4584,"1= on land, non-sensitive area","1= on land, non-sensitive area",1,0,0,0,0,0,0
7681,Land--Sensitive Area,2 land--sensitive area,0,1,0,0,0,0,0
3791,1,1,1,0,0,0,0,0,0
2133,1,1,1,0,0,0,0,0,0


In [28]:
# DONE
col_name = 'spill_on_water_edit'

typo_corrections = {
    '?': 'unknown',
    'undetermined': 'unknown'
}

target_labels = [
]

col_threshold = 90

# Cleaning process
new_col_name = col_name + '_clean'
data[new_col_name] = impute_nulls(data[col_name]).apply(lambda x: x.lower())
data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))
data[new_col_name].value_counts()

spill_on_water_edit_clean
no                  9199
null                 765
yes                  351
unknown                5
1                      2
sulfide reducer        1
paraffin               1
rainwater              1
isolated puddles       1
Name: count, dtype: int64

In [30]:
data['operator_edit'].value_counts()

operator_edit
Occidental Petroleum                                   997
Pioneer Natural Resources (acq. by ExxonMobil 2023)    596
XTO Energy (ExxonMobil subsidiary)                     589
Endeavor Energy                                        393
Energen (acq. by Diamondback 2019)                     310
                                                      ... 
HUGOTON OPERATING COMPANY, INC                           1
OILTON RESOURCES, INC                                    1
M.E. OPERATING & SERVICES                                1
FENOGLIO OIL COMPANY                                     1
PINON OPERATING LLC                                      1
Name: count, Length: 1160, dtype: int64

In [31]:
data['operator_edit'].value_counts().head(60)

operator_edit
Occidental Petroleum                                   997
Pioneer Natural Resources (acq. by ExxonMobil 2023)    596
XTO Energy (ExxonMobil subsidiary)                     589
Endeavor Energy                                        393
Energen (acq. by Diamondback 2019)                     310
Apache                                                 287
Anadarko (acq. by Occidental 2019)                     235
Parsley Energy                                         208
Diamondback                                            179
Denbury                                                178
Linn Operating                                         171
Chesapeake Energy                                      153
Crownquest Operating                                   145
EOG Resources                                          144
Laredo Petroleum                                       142
COG Operating                                          135
Kinder Morgan                             

In [32]:
# See how much collapse occurs when you lowercase things - nice
print(data['operator_edit'].nunique())
print(data['operator_edit'].str.lower().nunique())

1160
1138


In [33]:
# What percentage of these strings only have a value count of 1?
value_counts = data['operator_edit'].value_counts()
duplicated_values = value_counts[value_counts == 1].index.tolist()
print(len(duplicated_values))

print(value_counts.describe())

661
count    1160.000000
mean        8.900862
std        45.697486
min         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
max       997.000000
Name: count, dtype: float64


In [34]:
# DONE
col_name = 'lease_facility_name'

typo_corrections = {
    '?': 'unknown',
    'undetermined': 'unknown'
}

target_labels = [
]

col_threshold = 90

# Cleaning process
new_col_name = col_name + '_clean'
data[new_col_name] = impute_nulls(data[col_name]).apply(lambda x: x.lower())
data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))
data[new_col_name].value_counts()

lease_facility_name_clean
fullerton clearfork unit    124
sacroc unit                 107
denver unit                  79
means/san andres/unit        74
null                         72
                           ... 
o'daniel, e.t.                1
bednorz 38                    1
sale ranch 25c                1
monroe 39 'a'                 1
huey, sally                   1
Name: count, Length: 6547, dtype: int64

In [40]:
data['lease_facility_name'].value_counts()

lease_facility_name
Fullerton Clearfork Unit    124
Sacroc Unit                 107
Denver Unit                  79
Means/San Andres/Unit        74
Levelland Unit               65
                           ... 
Powell "8"                    1
Pecos Shearer Unit            1
Tom A                         1
Koger E 10                    1
Huey, Sally                   1
Name: count, Length: 6651, dtype: int64

In [44]:
# TODO
data['lease_facility_name'].sort_values().head(300).tolist()

['00128 pipeline t4 permit.',
 '10"  Pipeline - Kilgore City',
 '10" Line Hopkins Facility',
 '10" Line SND Burnside to McGeorge Tank',
 '10" PVC Line @ 32.5011, -94.86428',
 '115 SWD',
 '16" BIGFOOT SOUR PIPELINE',
 '1893 Oil & Gas Ltd.,',
 '21601 Radio Tower State',
 '281 SWD',
 '281 SWD',
 '281 SWD',
 '283 SWD',
 '3042 Heard Ranch AS PL',
 '32.555789, -94.794136',
 '508-32-Delmita Gathering System',
 '6 Pounder SE',
 '6" Deacon, Jim SDR 11',
 '6" Gravity Line-Cates Ln',
 '6" Grvity Line @ 32.4909, -94.9153',
 '6" LINE',
 '6" Polyline',
 '6" Spence Gathering Line',
 '6" TOTAL FLUIDS LINE',
 '6" gravity line - Harris Lease',
 '651 LTD. Benedum Univ.',
 '7203 JV-S Amoco',
 '7306 JV-S Wink',
 '7502 JV-S R.O.C.',
 '8 5/8" Gas Gather Line',
 '8" Fiberglass transfer line',
 '8" GATHERING LINE, Becker HG GU',
 '8" Poly Line',
 '8" Quantum/Castleberry SW line',
 '8" Rosanky to Nixon Pipeline',
 '8" SWPSI LINE FROM ANDERSON/ENGLAND',
 '8" Transite Line',
 '8" line on king lease',
 '8-in Gathe

## Cleaning Rep Cols

In [45]:
rep_cols = [
'date_called_in_edit',
'date_of_spill_edit',
'spill_number', 
'rrc_job_number',
'basis', 
'affected_area', 
'swr_20', 
'swr_98exempt',
'cleanup_criteria', 
'cleanup_criteria_swr_91', 
'cleanup_criteria_7_00_doc', 
'cleanup_criteria_case_specific', 
'form_h_8rqrd_edit',
'date_h_8rcvd', 
'cleanup_oversight', 
'cleanup_oversight_district', 
'cleanup_oversight_austin', 
'status', 
'compliance_date'
]
print(len(rep_cols))

19


In [46]:
complete_log = np.zeros(19)
for col, complete in zip(rep_cols, complete_log): 
    print(col)
    print('sum of nulls:', data[col].isnull().sum())
    print('coverage:', ((len(data)-data[col].isnull().sum())/ len(data)) * 100)
    print('unique values:', data[col].nunique())
    print(data[col].value_counts())
    if complete == 1: 
        print('!!! COMPLETE !!!')
    print()
    print()

date_called_in_edit
sum of nulls: 24
coverage: 99.76757699012202
unique values: 2948
date_called_in_edit
2015-10-21    22
2022-12-27    21
2015-01-05    19
2016-07-11    18
2021-02-23    18
              ..
2019-02-07     1
2019-02-05     1
2019-03-24     1
2019-03-19     1
2021-02-03     1
Name: count, Length: 2948, dtype: int64


date_of_spill_edit
sum of nulls: 0
coverage: 100.0
unique values: 3287
date_of_spill_edit
2015-05-19    16
2015-10-20    16
2017-08-27    14
2015-01-06    14
2015-01-15    14
              ..
2020-12-04     1
2020-12-08     1
2020-12-31     1
2021-12-25     1
2021-02-03     1
Name: count, Length: 3287, dtype: int64


spill_number
sum of nulls: 2876
coverage: 72.14797598295564
unique values: 6821
spill_number
-           25
9            8
5            8
37           7
29           7
            ..
08-13582     1
08-13580     1
08-13578     1
08-13569     1
8A-14592     1
Name: count, Length: 6821, dtype: int64


rrc_job_number
sum of nulls: 24
coverage: 99.76

In [51]:
# Column 
col_name = 'basis'

typo_corrections = {
    'opreator': 'operator'

}
new_col_name = col_name + '_clean'

# Cleaning process
data[new_col_name] = data[col_name].astype(str)
data[new_col_name] = impute_nulls(data[new_col_name]).apply(lambda x: x.lower())
data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))

# Check discrepancies
discrepancies = data[[col_name, new_col_name]]
discrepancies = discrepancies[discrepancies[col_name] != discrepancies[new_col_name]]
discrepancies.drop_duplicates()

,basis,basis_clean
0,Other Rpt,other rpt
1,NaN,nan
11,H-8,h-8
128,Verbal,verbal
205,Letter,letter
341,Rule 20,rule 20
612,SWR 20 Letter,swr 20 letter
1004,other Rpt,other rpt
3806,VERBAL,verbal
3810,EMAIL,email


In [52]:
data[new_col_name].value_counts()

basis_clean
other rpt        4320
nan              2070
operator         1681
h-8              1283
0                 668
verbal            236
email              39
other              14
letter              5
other report        4
?                   2
rule 20             1
swr 20 letter       1
80                  1
smcu                1
Name: count, dtype: int64

In [53]:
# Column 
col_name = 'swr_20'

typo_corrections = {
    'opreator': 'operator'
}

new_col_name = col_name + '_clean'

# Cleaning process
data[new_col_name] = data[col_name].astype(str)
data[new_col_name] = impute_nulls(data[new_col_name]).apply(lambda x: x.lower())
data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))

# Check discrepancies
discrepancies = data[[col_name, new_col_name]]
discrepancies = discrepancies[discrepancies[col_name] != discrepancies[new_col_name]]
discrepancies.drop_duplicates()

,swr_20,swr_20_clean
1,NaN,nan
128,Yes,yes
1470,No,no
3806,NO,no
3807,YES,yes
6793,NR,nr


In [54]:
data[new_col_name].value_counts()

swr_20_clean
yes                   6728
no                    2461
0                      708
nan                    426
nr                       2
yes (not received)       1
Name: count, dtype: int64

In [55]:
# Column 
col_name = 'swr_98exempt'

typo_corrections = {
}

new_col_name = col_name + '_clean'

# Cleaning process
data[new_col_name] = data[col_name].astype(str)
data[new_col_name] = impute_nulls(data[new_col_name]).apply(lambda x: x.lower())
data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))

# Check discrepancies
discrepancies = data[[col_name, new_col_name]]
discrepancies = discrepancies[discrepancies[col_name] != discrepancies[new_col_name]]
discrepancies.drop_duplicates()

,swr_98exempt,swr_98exempt_clean
1,NaN,nan
128,No,no
3155,YES,yes
3806,NO,no
4075,no (lined FW),no (lined fw)
5978,Yes,yes


In [56]:
data[new_col_name].value_counts()

swr_98exempt_clean
no               9110
0                 709
nan               359
yes               147
no (lined fw)       1
Name: count, dtype: int64

In [57]:
# Column 
col_name = 'cleanup_criteria'

typo_corrections = {
    'swr91': 'swr 91',

}

new_col_name = col_name + '_clean'

# Cleaning process
data[new_col_name] = data[col_name].astype(str)
data[new_col_name] = impute_nulls(data[new_col_name]).apply(lambda x: x.lower())
data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))

# Check discrepancies
discrepancies = data[[col_name, new_col_name]]
discrepancies = discrepancies[discrepancies[col_name] != discrepancies[new_col_name]]
discrepancies.drop_duplicates()

,cleanup_criteria,cleanup_criteria_clean
0,NaN,nan
7294,SWR 91,swr 91
7295,Case Specific,case specific
7600,No,no
7601,Yes,yes
7613,Condensate Guidance,condensate guidance
8063,SWR 92,swr 92
8064,SWR 93,swr 93
8373,SWR91,swr 91
8722,sWR 91,swr 91


In [58]:
data[new_col_name].value_counts()

cleanup_criteria_clean
nan                    8075
swr 91                 1626
case specific           590
condensate guidance      27
no                        4
yes                       2
swr 92                    1
swr 93                    1
Name: count, dtype: int64

In [59]:
# Column 
col_name = 'cleanup_criteria_swr_91'

typo_corrections = {
    'swr91': 'swr 91',

}
new_col_name = col_name + '_clean'

# Cleaning process
data[new_col_name] = data[col_name].astype(str)
data[new_col_name] = impute_nulls(data[new_col_name]).apply(lambda x: x.lower())
data[new_col_name] = data[new_col_name].apply(lambda x: correct_typos(x, typo_corrections))

# Check discrepancies
discrepancies = data[[col_name, new_col_name]]
discrepancies = discrepancies[discrepancies[col_name] != discrepancies[new_col_name]]
discrepancies.drop_duplicates()

,cleanup_criteria_swr_91,cleanup_criteria_swr_91_clean
1,NaN,nan
2,Yes,yes
72,No,no
217,YES,yes
764,X,x
3809,NO,no


In [60]:
data[new_col_name].value_counts()

cleanup_criteria_swr_91_clean
nan    3753
yes    3508
no     2901
x       146
0        15
133       1
120       1
107       1
Name: count, dtype: int64

In [61]:
data.to_csv('./clean_data/' + 'clean_central_data.csv')